# Top Pages (Last 7 Days)

This notebook queries GA4 export data in BigQuery and shows which pages received the most traffic in the last 7 days.

**Project:** `lifeline-website-480522`  
**Dataset:** `analytics_315584957`

In [1]:
from google.cloud import bigquery
import pandas as pd
import plotly.express as px

PROJECT_ID = "lifeline-website-480522"
DATASET = "analytics_315584957"
TOP_N = 15

client = bigquery.Client(project=PROJECT_ID)

/home/aido/projects/lla-data/.venv/lib/python3.14/site-packages/google/auth/_default.py:114: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [2]:
import sys
sys.path.insert(0, "..")
import lifeline_theme  # registers the "lifeline" Plotly template

lifeline_theme.inject_fonts()  # loads Open Sans from Google Fonts into this notebook

In [3]:
query = f"""
SELECT
  COALESCE((
    SELECT ep.value.string_value
    FROM UNNEST(event_params) ep
    WHERE ep.key = 'page_location'
  ), '(unknown)') AS page_location,
  COUNT(*) AS page_views
FROM `{PROJECT_ID}.{DATASET}.events_*`
WHERE event_name = 'page_view'
  AND _TABLE_SUFFIX BETWEEN FORMAT_DATE('%Y%m%d', DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY))
  AND FORMAT_DATE('%Y%m%d', CURRENT_DATE())
GROUP BY page_location
ORDER BY page_views DESC
LIMIT {TOP_N}
"""

job = client.query(query)
df = job.to_dataframe()
df

/home/aido/projects/lla-data/.venv/lib/python3.14/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,page_location,page_views
0,https://www.lifeline.org.au/,12577
1,https://www.lifeline.org.au/chat,4465
2,https://www.lifeline.org.au/get-help/national-...,4323
3,https://www.lifeline.org.au/chat?q=crisis-chat/,3613
4,https://www.lifeline.org.au/search?search=depr...,3390
5,https://www.outoftheshadowswalk.org.au/tribute...,3203
6,https://www.lifeline.org.au/get-help/support-t...,2103
7,https://www.lifeline.org.au/chat/staging-chat,2023
8,https://www.lifeline.org.au/chat?q=get-help/se...,1947
9,https://www.lifeline.org.au/get-help/support-t...,914


In [4]:
df["page_path"] = lifeline_theme.strip_domain(df["page_location"])

fig = px.bar(
    df.sort_values("page_views"),          # ascending so top pages appear at the top
    x="page_views",
    y="page_path",
    orientation="h",
    title="Top Pages by Page Views (Last 7 Days)",
    labels={"page_path": "Page", "page_views": "Page Views"},
    template="lifeline",
)
# Wide left margin to accommodate full path labels
fig.update_layout(height=550, yaxis_title=None, margin=dict(l=260))
lifeline_theme.add_lifeline_logo(fig)
fig.show()

If you get an auth error, run this once in terminal:

```bash
gcloud auth application-default login
```